# Task 2: Build a Q&A application using Amazon Bedrock Knowledge Bases with Retrieve API

In this task, you build a Q&A application using Amazon Bedrock Knowledge Bases with *Retrieve* API. Here, you query the knowledge base to get the desired number of document chunks based on similarity search. You then augment the prompt with relevant documents and perform a query which acts as input to Amazon Nova Lite for generating response.

With a knowledge base, you can securely connect foundation models (FMs) in Amazon Bedrock to your company's data for Retrieval Augmented Generation (RAG). Access to additional data helps the model generate more relevant, context-specific and accurate responses without continuously retraining the FM. All information retrieved from knowledge bases comes with source attribution to improve transparency and minimize hallucinations.

<i aria-hidden="true" class="fas fa-info-circle" style="color:#007FAA"></i> **Learn more:** For more information on creating a knowledge base using console, refer to *[Amazon Bedrock Knowledge Bases](https://docs.aws.amazon.com/bedrock/latest/userguide/knowledge-base.html)*.

### Scenario

You implement the solution using Retrieval Augmented Generation (RAG) pattern. RAG retrieves data from outside the language model and augments the prompts by adding the retrieved data in context. Here, you are performing RAG effectively on the knowledge base created as part of lab provisioning.
    
In this notebook you:

- Use AnyCompany's financial 10k reports (synthetically generated dataset) as a text corpus to perform Q&A on. This data is already ingested into the Amazon Bedrock Knowledge Bases during lab provisioning.
- Use the Knowledge Base ID for an existing knowledge base that was created for this lab environment.
- Use both Amazon Bedrock *Retrieve API* and LangChain retrieval to retrieve documents form the knowledge base, and add that as context to answer user queries.

<i aria-hidden="true" class="fas fa-exclamation-circle" style="color:#7C5AED"></i> **Caution:** It is recommended to run each code cell individually rather than using the **Run All Cells** option from the **Run** menu. Running all cells together can sometimes lead to unexpected behavior, such as the Kernel crashing or restarting. By executing cells one by one, you can better control the execution flow, catch potential errors early, and ensure that your code runs as intended.

### Task 2.1: Setup the environment

In this task, you initiate the Amazon Bedrock client and perform the following operations:

- Verify the Knowledge Base ID.
- Import the necessary libraries and set up the necessary clients.

#### Task 2.1.1: Verify the Knowledge Base ID

To run this notebook, you need to verify and assign the Knowledge Base ID to the *kb_id* variable and install the required packages.

1. Run the following code cell to verify the ID for the existing Knowledge Base in Amazon Bedrock:

In [1]:
import boto3
import botocore

session = boto3.Session()
bedrock_client = session.client('bedrock-agent')

try:
    response = bedrock_client.list_knowledge_bases(
        maxResults=1  # We only need to retrieve the first Knowledge Base
    )
    knowledge_base_summaries = response.get('knowledgeBaseSummaries', [])

    if knowledge_base_summaries:
        kb_id = knowledge_base_summaries[0]['knowledgeBaseId']
        print(f"Knowledge Base ID: {kb_id}")
    else:
        print("No Knowledge Base summaries found.")
        
except botocore.exceptions.ClientError as e:
    print(f"Error: {e}")

####  Task 2.1.2: Initiate the Amazon Bedrock client

2. Run the following code cell to import the necessary libraries for setting up your environment:

In [ ]:
import boto3
from botocore.client import Config
import pprint
import json

pp = pprint.PrettyPrinter(indent=2)

session = boto3.session.Session()
region = session.region_name

bedrock_config = Config(connect_timeout=120, read_timeout=120, retries={'max_attempts': 0})
bedrock_client = boto3.client('bedrock-runtime', region_name = region)
bedrock_agent_client = boto3.client("bedrock-agent-runtime",
                              config=bedrock_config, region_name = region)

## Part 1: Use *Retrieve* API with foundation models from Amazon Bedrock

<i aria-hidden="true" class="fas fa-sticky-note" style="color:#563377"></i> **Note:** You use the *amazon.nova-lite-v1:0* model for this part.

### Task 2.2: Use Retrieve API with foundation models from Amazon Bedrock

In this task, you define a retrieve function that calls the *Retrieve* API provided by the Knowledge Base for Amazon Bedrock which converts user queries into embeddings, searches the knowledge base, and returns the relevant results, giving you more control to build custom workflows on top of the semantic search results.

The output of the *Retrieve* API includes the *retrieved text chunks*, the *location type* and *URI* of the source data, as well as the relevance *scores* of the retrievals. You can also use the *overrideSearchType* option in *retrievalConfiguration* which offers the choice to use either *HYBRID* or *SEMANTIC*.

By default, it selects the right strategy for you to give you most relevant results. If you want to override the default option to use either hybrid or semantic search, you can set the value to *HYBRID/SEMANTIC*.

<!-- ![retrieveAPI](./images/retrieveAPI.png) -->
<img src="images/retrieveAPI.png" width=50% height=20% />

*Image description: The preceding diagram depicts the customized RAG workflow for the lab environment.*

3. Run the following code cell to define a *retrieve* function that calls the *Retrieve* API:

In [3]:
def retrieve(query, kbId, numberOfResults=5):
    return bedrock_agent_client.retrieve(
        retrievalQuery= {
            'text': query
        },
        knowledgeBaseId=kbId,
        retrievalConfiguration= {
            'vectorSearchConfiguration': {
                'numberOfResults': numberOfResults,
                'overrideSearchType': "HYBRID", # optional
            }
        }
    )

#### Task 2.2.1: Initialize your Knowledge Base ID before querying responses from the initialized LLM

In this task, you call the *Retrieve* API, and pass the *Knowledge Base ID*, *number of results* and *query* as parameters. 

<i aria-hidden="true" class="fas fa-sticky-note" style="color:#563377"></i> **Note:** You can view the associated score of each of the text chunk that was returned which depicts its correlation to the query in terms of how closely it matches it.

4. Run the following code cell to call the *Retrieve* API by passing the *Knowledge Base ID*, *number of results* and *query* as parameters:

In [ ]:
query = "What was the total operating lease liabilities and total sublease income of the AnyCompany as of December 31, 2022?"
response = retrieve(query, kb_id, 5)
retrievalResults = response['retrievalResults']
pp.pprint(retrievalResults)

#### Task 2.2.2: Extract the text chunks from the *Retrieve* API response

In this task, you extract the text chunks from the *Retrieve* API response.

5. Run the following two code cells to fetch the context from the retrieval results and print them:

In [5]:
# fetch context from the response
def get_contexts(retrievalResults):
    contexts = []
    for retrievedResult in retrievalResults: 
        contexts.append(retrievedResult['content']['text'])
    return contexts

In [ ]:
contexts = get_contexts(retrievalResults)
pp.pprint(contexts)

#### Task 2.2.3: Use specific prompt for the model to generate personalized responses 

In the task, you use a specific prompt for the model to act as a financial advisor AI system that provides answers to questions by using fact based and statistical information when possible. You provide the *Retrieve* API responses from earlier task as a part of the *{contexts}* in the prompt for the model to refer to, along with the user *query*.

6. Run the following code cell to use a specific prompt for the model to act as a financial advisor AI system:

In [7]:
prompt = f"""
Human: You are a financial advisor AI system, and provides answers to questions by using fact based and statistical information when possible. 
Use the following pieces of information to provide a concise answer to the question enclosed in <question> tags. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
<context>
{contexts}
</context>

<question>
{query}
</question>

The response should be specific and use statistics or numbers when possible.

Assistant:"""

#### Task 2.2.4: Invoke foundation model from Amazon Bedrock

In this task, you use the **amazon.nova-lite-v1:0** foundation model from Amazon Bedrock.

7. Run the following two code cells to invoke the **amazon.nova-lite-v1:0** foundation model from Amazon Bedrock. You are passing both the context and the query to the model:

In [ ]:
# payload with model parameters
messages = [{
    "role": "user",
    "content": [{"text": prompt}]
}]

# Create the proper Nova Lite payload
nova_payload = {
    "schemaVersion": "messages-v1",
    "messages": messages,
    "inferenceConfig": {
        "maxTokens": 512,
        "temperature": 0.5,
        "topP": 0.9,
        "topK": 20
    }
}

In [ ]:
modelId = 'amazon.nova-lite-v1:0' # change this to use a different version from the model provider
accept = 'application/json'
contentType = 'application/json'

response = bedrock_client.invoke_model(
    body=json.dumps(nova_payload),
    modelId=modelId,
    accept=accept,
    contentType=contentType
)

# Parse and extract the response
response_body = json.loads(response.get('body').read())

# Extract just the text from the response
response_text = ''
if 'output' in response_body and 'message' in response_body['output']:
    message_content = response_body['output']['message']['content']
    if message_content and isinstance(message_content, list):
        response_text = message_content[0].get('text', '')

# Print the response text
print(response_text)

## Part 2: LangChain integration

### Task 2.3: LangChain integration

In this task, you build a Q&A application using the *AmazonKnowledgeBasesRetriever* class from LangChain. You query the knowledge base to get the desired number of document chunks based on similarity search. Following that you integrate it with LangChain chain to pass the document chunks and the query to the llm (**Amazon Nova Lite**) for answering questions.

#### Task 2.3.1: Environment setup

In this task, you set up your environment.

8. Run the following code cell to import the necessary packages for setting up your environment:

In [ ]:
import langchain
from langchain_aws import ChatBedrock
from langchain_aws.retrievers.bedrock import AmazonKnowledgeBasesRetriever

llm = ChatBedrock(model_id=modelId, 
                  client=bedrock_client)

#### Task 2.3.2: Create AmazonKnowledgeBasesRetriever object that calls the Retrieve API

In this task, you create a *AmazonKnowledgeBasesRetriever* object from LangChain which calls the *Retrieve* API provided by Amazon Bedrock Knowledge Bases. This converts user queries into embeddings, searches the knowledge base, and returns the relevant results, giving you more control to build custom workflows on top of the semantic search results.

9. Run the following code cell to create the *AmazonKnowledgeBasesRetriever* object:

In [ ]:
query = "What was the total operating lease liabilities and total sublease income of the AnyCompany as of December 31, 2022?"
retriever = AmazonKnowledgeBasesRetriever(
        knowledge_base_id=kb_id,
        retrieval_config={"vectorSearchConfiguration": 
                          {"numberOfResults": 4,
                           'overrideSearchType': "SEMANTIC", # optional
                           }
                          },
        # endpoint_url=endpoint_url,
        # region_name=region,
        # credentials_profile_name="<profile_name>",
    )
docs = retriever.invoke(
        input=query
    )
for doc in docs:
    print(doc.page_content)
    print("------")

#### Task 2.3.3: Use specific prompt to the model to get personalized responses

In this task, you use specific prompt for the model to act as a financial advisor AI system that provides answers to questions by using fact based and statistical information when possible. You provide the *Retrieve* API responses from above as a part of the *{context}* in the prompt for the model to refer to, along with the user *query*.

10. Run the following code cell to use a specific prompt for the model to act as a financial advisor AI system:

In [ ]:
from langchain.prompts import PromptTemplate

PROMPT_TEMPLATE = """
Human: You are a financial advisor AI system, and provides answers to questions by using fact based and statistical information when possible. 
Use the following pieces of information to provide a concise answer to the question enclosed in <question> tags. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
<context>
{context}
</context>

<question>
{question}
</question>

The response should be specific and use statistics or numbers when possible.

Assistant:"""
nova_prompt = PromptTemplate(template=PROMPT_TEMPLATE, 
                               input_variables=["context","question"])

#### Task 2.3.4: Integrate the retriever and the LLM with retrieval chain to build the Q&A application

In this task, you integrate the retriever and the LLM using Langchain Expression Language (LCEL) to build the Q&A application.

11. Run the following cell to integrate the retriever and the LLM with the documents retrieved using retriever.invoke. Print the results:

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

def format_docs(docs): #concatenate the text from the page_content field in the output from retriever.invoke
    return "\n\n".join(doc.page_content for doc in docs)

chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | nova_prompt
    | llm
    | StrOutputParser()
)

response=chain.invoke(query)
print(response)

<i aria-hidden="true" class="far fa-thumbs-up" style="color:#008296"></i> **Task complete:** You have completed this notebook. To move to the next part of the lab, do the following:

- Close this notebook file.
- Return to the lab session and continue with Task 3.